<a href="https://colab.research.google.com/github/pstorniolo/Master2021/blob/main/2021_10_23_Spark_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DataFrame

In [ ]:
# Install Spark 3.2.0
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!rm -f *.tgz*

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

!pip install -q findspark
!pip install -q pyspark==3.2.0

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row

from datetime import datetime, date
import pandas as pd

spark=SparkSession.builder.appName("local[*]").getOrCreate()

##Create a PySpark DataFrame from a list of rows.

In [ ]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])

print(df)

In [ ]:
df.show()

##Create a PySpark DataFrame with an explicit schema.

In [ ]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a bigint, b double, c string, d date, e timestamp')

print(df)

##Create a PySpark DataFrame from a pandas DataFrame

In [ ]:
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
print(pandas_df)

df = spark.createDataFrame(pandas_df)
print(df)

In [ ]:
df.show()

## Read data with Pandas

In [ ]:
!ls -la sample_data

### CSV file

In [ ]:
pandas_df = pd.read_csv('sample_data/california_housing_train.csv')

california_housing_df = spark.createDataFrame(pandas_df)

california_housing_df.show()
california_housing_df.printSchema()

In [ ]:
!cat sample_data/anscombe.json

###JSON file

In [ ]:
pandas_df = pd.read_json('sample_data/anscombe.json')

print(pandas_df)
anscombe_df = spark.createDataFrame(pandas_df)
anscombe_df.show()
anscombe_df.printSchema()

In [ ]:
anscombe_df.show(2, vertical=True)

In [ ]:
anscombe_df.columns

In [ ]:
anscombe_df.take(3)

In [ ]:
california_housing_df.take(3)

##Selecting and Accessing Data

PySpark DataFrame viene valutato in "modo minimale" e la semplice selezione di una colonna non attiva un calcolo ma restituisce un'istanza di colonna.


In [ ]:
df.a
df.printSchema()

La maggior parte delle operazioni per colonna restituisce colonne.

In [ ]:
from pyspark.sql import Column
from pyspark.sql.functions import upper

type(df.c) == type(upper(df.c)) == type(df.c.isNull())

Queste colonne possono essere utilizzate per selezionare le colonne da un DataFrame. Per esempio, **DataFrame.select()** accetta le istanze di *Column* e restituisce un altro DataFrame.

In [ ]:
df.select(df.c).show()

Assegna una nuova istanza di colonna.

In [ ]:
df.withColumn('upperC', upper(df.c)).show()

In [ ]:
df.withColumn('C', upper(df.c)).show()

In [ ]:
df.withColumn('B', upper(df.c)).show()

Per selezionare un sottoinsieme di righe, si utilizza **DataFrame.filter()**.

In [ ]:
df.filter(df.a == 1).show()

In [ ]:
df.filter(df.a != 1).show()

##Applying a Function

PySpark supporta varie UDF (*user-defined function*) e API per consentire agli utenti di eseguire funzioni native di Python.L'esempio seguente consente agli utenti di utilizzare direttamente le API in una serie di panda all'interno della funzione nativa di Python.

In [ ]:
import pandas
from pyspark.sql.functions import pandas_udf

@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1

df.select(pandas_plus_one(df.a)).show()

Un altro esempio è **DataFrame.mapInPandas** che consente agli utenti di utilizzare direttamente le API in un DataFrame panda senza alcuna restrizione come la lunghezza del risultato.

In [ ]:
def pandas_filter_func(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.a == 1]

df.mapInPandas(pandas_filter_func, schema=df.schema).show()

In [ ]:
def pandas_filter_func(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.a != 1]

df.mapInPandas(pandas_filter_func, schema=df.schema).show()

##Grouping Data

PySpark DataFrame fornisce anche un modo per gestire i dati raggruppati utilizzando l'approccio comune, la strategia *split-apply-combine*. Raggruppa i dati in base a una determinata condizione applica una funzione a ciascun gruppo e quindi li combina di nuovo al DataFrame.

In [ ]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], 
    schema=['color', 'fruit', 'v1', 'v2'])
df.show()

Raggruppamento e quindi applicazione della funzione **avg()** ai gruppi risultanti.

In [ ]:
df.printSchema()

In [ ]:
df.groupby('color').avg().show()

In [ ]:
df.groupby('fruit').avg().show()

Si può applicare anche una funzione nativa Python a ciascun gruppo utilizzando l'API pandas.

In [ ]:
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 - pandas_df.v1.mean())

df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

Co-raggruppamento e applicazione di una funzione.

---
`pandas.merge_asof(left, right, on=None, left_on=None, right_on=None, left_index=False, right_index=False, by=None, left_by=None, right_by=None, suffixes=('_x', '_y'), tolerance=None, allow_exact_matches=True, direction='backward')`

---
L'unione asof è simile a un left-join tranne per il fatto che abbiniamo la chiave più vicina anziché le chiavi uguali.


In [ ]:
df1 = spark.createDataFrame(
    [(20000101, 1, 1.0), (20000101, 2, 2.0), (20000102, 1, 3.0), (20000102, 2, 4.0)],
    ('time', 'id', 'v1'))
df1.show()

df2 = spark.createDataFrame(
    [(20000101, 1, 'x'), (20000101, 2, 'y')],
    ('time', 'id', 'v2'))
df2.show()

def asof_join(l, r):
    return pd.merge_asof(l, r, on='time', by='id')

df1.groupby('id').cogroup(df2.groupby('id')).applyInPandas(
    asof_join, schema='time int, id int, v1 double, v2 string').show()

In [ ]:
df1.printSchema()
df2.printSchema()

##Getting Data in/out

In [ ]:
!rm -rf foo*

df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], 
    schema=['color', 'fruit', 'v1', 'v2'])
df.show()

###Read & Write **CSV**

In [ ]:
df.write.csv('foo.csv')
spark.read.csv('foo.csv').show()

In [ ]:
!rm -rf foo*
df.write.csv('foo.csv', header=True)
spark.read.csv('foo.csv', header=True).show()

In [ ]:
!ls -al foo.csv/

In [ ]:
!cat foo.csv/part-00000-bc27a3ed-ea6a-48bb-89af-e8fecbdc72a3-c000.csv

In [ ]:
!cat foo.csv/part-00001-bc27a3ed-ea6a-48bb-89af-e8fecbdc72a3-c000.csv

###Write & Read **Parquet**

In [ ]:
df.write.parquet('foo.parquet')
spark.read.parquet('foo.parquet').show()

In [ ]:
!ls -alh foo.parquet/

###Read & Write **ORC**

In [ ]:
df.write.orc('foo.orc')
spark.read.orc('foo.orc').show()

In [ ]:
!ls -la foo*

In [ ]:
pandas_df = df.toPandas()
print(pandas_df)
pandas_df.to_csv('foo_pandas.csv', index=False, header=True)

In [ ]:
!ls -la foo_pandas.csv
!cat foo_pandas.csv

In [ ]:
pandas_df = df.toPandas()
print(pandas_df)
pandas_df.to_csv('foo_pandas.csv', index=True, header=True)

In [ ]:
!cat foo_pandas.csv

In [ ]:
p_df=pd.read_csv('foo_pandas.csv', index_col=0)
print(p_df)

In [ ]:
spark.createDataFrame(p_df).show()

##Working with SQL

DataFrame e Spark SQL condividono lo stesso motore di esecuzione in modo che possano essere utilizzati in modo intercambiabile senza problemi. Ad esempio, si può registrare un DataFrame come tabella ed eseguire facilmente una query SQL:

In [ ]:
df.show()

In [ ]:
df.createOrReplaceTempView("tableA")
df.printSchema()
spark.sql("SELECT count(*) from tableA").show()

In [ ]:
spark.sql("SELECT fruit, count(*) from tableA group by fruit").show()

Inoltre, le UDF possono essere registrate e richiamate in SQL immediatamente:

In [ ]:
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
    return s + 1

spark.udf.register("add_one", add_one)
spark.sql("SELECT color, fruit, v1, add_one(v1) FROM tableA").show()

Queste espressioni SQL possono essere mescolate direttamente e utilizzate come colonne PySpark.

In [ ]:
from pyspark.sql.functions import expr

df.selectExpr('add_one(v1)').show()

In [ ]:
df.select(expr('count(*)') > 0).show()